# Predicting UFC Winners

## Overview

## Business Problem

## Data Understanding

In [55]:
# Importing everything needed
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import (accuracy_score, precision_score, ConfusionMatrixDisplay, 
recall_score, confusion_matrix, precision_score, make_scorer)
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA

In [4]:
# Read in the data and display a preview
df = pd.read_csv('data/ufc-master.csv')
df.head()

,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,finish_details,finish_round,finish_round_time,total_fight_time_secs,r_dec_odds,b_dec_odds,r_sub_odds,b_sub_odds,r_ko_odds,b_ko_odds
0,Thiago Santos,Johnny Walker,-150.0,130,66.666667,130.000000,2021-10-02,"Las Vegas, Nevada, USA",USA,Red,...,NaN,5.0,5:00,1500.0,800.0,900.0,2000.0,1600.0,-110.0,175.0
1,Alex Oliveira,Niko Price,170.0,-200,170.000000,50.000000,2021-10-02,"Las Vegas, Nevada, USA",USA,Blue,...,NaN,3.0,5:00,900.0,450.0,350.0,700.0,1100.0,550.0,120.0
2,Misha Cirkunov,Krzysztof Jotko,110.0,-130,110.000000,76.923077,2021-10-02,"Las Vegas, Nevada, USA",USA,Blue,...,NaN,3.0,5:00,900.0,550.0,275.0,275.0,1400.0,600.0,185.0
3,Alexander Hernandez,Mike Breeden,-675.0,475,14.814815,475.000000,2021-10-02,"Las Vegas, Nevada, USA",USA,Red,...,Punch,1.0,1:20,80.0,175.0,900.0,500.0,3500.0,110.0,1100.0
4,Joe Solecki,Jared Gordon,-135.0,115,74.074074,115.000000,2021-10-02,"Las Vegas, Nevada, USA",USA,Blue,...,NaN,3.0,5:00,900.0,165.0,200.0,400.0,1200.0,900.0,600.0


In [5]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4896 entries, 0 to 4895
Data columns (total 119 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   R_fighter                     object 
 1   B_fighter                     object 
 2   R_odds                        float64
 3   B_odds                        int64  
 4   R_ev                          float64
 5   B_ev                          float64
 6   date                          object 
 7   location                      object 
 8   country                       object 
 9   Winner                        object 
 10  title_bout                    bool   
 11  weight_class                  object 
 12  gender                        object 
 13  no_of_rounds                  int64  
 14  B_current_lose_streak         int64  
 15  B_current_win_streak          int64  
 16  B_draw                        int64  
 17  B_avg_SIG_STR_landed          float64
 18  B_avg_SIG_STR_pct          

In [6]:
df.describe()

,R_odds,B_odds,R_ev,B_ev,no_of_rounds,B_current_lose_streak,B_current_win_streak,B_draw,B_avg_SIG_STR_landed,B_avg_SIG_STR_pct,...,B_Flyweight_rank,B_Pound-for-Pound_rank,finish_round,total_fight_time_secs,r_dec_odds,b_dec_odds,r_sub_odds,b_sub_odds,r_ko_odds,b_ko_odds
count,4895.000000,4896.000000,4895.000000,4896.000000,4896.000000,4896.000000,4896.000000,4896.000000,3966.000000,4131.000000,...,95.000000,35.000000,4274.000000,4274.000000,4093.000000,4077.000000,3847.000000,3835.000000,3847.000000,3834.000000
mean,-117.640449,66.030637,94.827397,167.083323,3.181985,0.477941,0.875408,0.010621,26.308553,0.444741,...,8.473684,9.485714,2.408049,652.313758,294.064745,416.544027,843.010138,1064.543155,514.231869,647.257173
std,268.881452,247.803928,82.843409,136.944643,0.571515,0.769386,1.311379,0.108333,20.935885,0.121332,...,4.259763,4.300283,0.996643,357.911423,230.583958,306.571299,550.126761,627.285034,413.622768,458.846643
min,-1700.000000,-1200.000000,5.882353,8.333333,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,2.000000,1.000000,5.000000,-440.000000,-200.000000,-370.000000,-1250.000000,-550.000000,-275.000000
25%,-255.000000,-145.000000,39.215686,68.965517,3.000000,0.000000,0.000000,0.000000,5.610000,0.387500,...,5.000000,5.000000,1.000000,297.000000,167.000000,225.000000,435.000000,590.000000,240.000000,325.000000
50%,-150.000000,130.000000,66.666667,130.000000,3.000000,0.000000,0.000000,0.000000,24.759615,0.450000,...,8.000000,10.000000,3.000000,900.000000,250.000000,349.000000,720.000000,975.000000,435.000000,548.500000
75%,126.500000,220.000000,126.500000,220.000000,3.000000,1.000000,1.000000,0.000000,39.075000,0.510000,...,12.000000,13.500000,3.000000,900.000000,400.000000,525.000000,1200.000000,1400.000000,700.000000,880.750000
max,775.000000,1300.000000,775.000000,1300.000000,5.000000,6.000000,12.000000,2.000000,154.000000,1.000000,...,15.000000,15.000000,5.000000,1500.000000,2200.000000,2600.000000,4665.000000,4785.000000,2675.000000,3200.000000


In [7]:
df.shape

(4896, 119)

## Data Preparation

I want to change date column to a datetime object but they are in different formats

In [8]:
df['date']

0       2021-10-02
1       2021-10-02
2       2021-10-02
3       2021-10-02
4       2021-10-02
           ...    
4891     3/21/2010
4892     3/21/2010
4893     3/21/2010
4894     3/21/2010
4895     3/21/2010
Name: date, Length: 4896, dtype: object

Needed to make sure all of them were separated by the same sign.

In [9]:
# Changing all / to - in date column
df['date'] = df['date'].str.replace('/', '-')

The first 58 dates were in a different format. They had year-month-day instead of month-day-year.

In [10]:
df['date'][:59]

0     2021-10-02
1     2021-10-02
2     2021-10-02
3     2021-10-02
4     2021-10-02
5     2021-10-02
6     2021-10-02
7     2021-10-02
8     2021-10-02
9     2021-10-02
10    2021-10-02
11    2021-09-25
12    2021-09-25
13    2021-09-25
14    2021-09-25
15    2021-09-25
16    2021-09-25
17    2021-09-25
18    2021-09-25
19    2021-09-25
20    2021-09-25
21    2021-09-25
22    2021-09-25
23    2021-09-25
24    2021-09-18
25    2021-09-18
26    2021-09-18
27    2021-09-18
28    2021-09-18
29    2021-09-18
30    2021-09-18
31    2021-09-18
32    2021-09-18
33    2021-09-18
34    2021-09-18
35    2021-09-18
36    2021-09-18
37    2021-09-04
38    2021-09-04
39    2021-09-04
40    2021-09-04
41    2021-09-04
42    2021-09-04
43    2021-09-04
44    2021-09-04
45    2021-09-04
46    2021-08-28
47    2021-08-28
48    2021-08-28
49    2021-08-28
50    2021-08-28
51    2021-08-28
52    2021-08-28
53    2021-08-28
54    2021-08-28
55    2021-08-28
56    2021-08-28
57    2021-08-28
58     8-21-20

Wrote a for loop that goes through each row in the `date` column and rearranges the strings by slicing. After this I was able to change the data type.

In [11]:
# Only want to change the first 58 values
for x in range(58):
    # Accessing each value in date column
    string = df.loc[x, 'date']
    # Slicing to get day, month, and year separated
    day = string[8:]
    month = string[5:7]
    year = string[:4]
    # Moving components around to get month-day-year format
    new_format = month + '-' + day + '-' + year
    # Update the DataFrame
    df.loc[x, 'date'] = new_format

Changed the data type to a datetime object

In [12]:
# Converting column to datetime from object
df['date'] = pd.to_datetime(df['date'], format='%m-%d-%Y')

After I changed the data type of the `date` column I decided to explore null values.

In [16]:
# Checking columns with nulls
df.isna().sum()

R_fighter        0
B_fighter        0
R_odds           1
B_odds           0
R_ev             1
              ... 
b_dec_odds     819
r_sub_odds    1049
b_sub_odds    1061
r_ko_odds     1049
b_ko_odds     1062
Length: 119, dtype: int64

There are a lot of nulls in the columns below because only 10 fighters in each weightclass are ranked at any given moment in time. This means majority of fighters do not have a rank. It is also rare for a nonranked fighter to fight a ranked fighter. I don't think these columns add value so I am comfortable dropping them and getting rid of all those null values they hold.

In [17]:
# Looking of list of columns with the most null values
nulls = df.isna().sum()
nulls.sort_values(ascending=False)[:30]

B_Women's Featherweight_rank    4896
R_Women's Featherweight_rank    4889
B_Pound-for-Pound_rank          4861
B_Women's Flyweight_rank        4852
R_Women's Flyweight_rank        4837
B_Women's Strawweight_rank      4835
B_Women's Bantamweight_rank     4818
B_Bantamweight_rank             4811
B_Lightweight_rank              4809
B_Welterweight_rank             4807
B_Featherweight_rank            4806
B_Light Heavyweight_rank        4803
B_Flyweight_rank                4801
B_Middleweight_rank             4794
R_Women's Strawweight_rank      4792
B_Heavyweight_rank              4786
R_Women's Bantamweight_rank     4778
R_Featherweight_rank            4763
R_Middleweight_rank             4762
R_Bantamweight_rank             4759
R_Lightweight_rank              4757
R_Welterweight_rank             4756
R_Light Heavyweight_rank        4755
R_Heavyweight_rank              4754
R_Flyweight_rank                4754
R_Pound-for-Pound_rank          4730
B_match_weightclass_rank        4019
R

I will also get rid of the column `finish_details` because it has over 1000 nulls.

In [18]:
# Creating list of all columns to drop
columns_to_drop = list(nulls.sort_values(ascending=False)[:29].index)
columns_to_drop

["B_Women's Featherweight_rank",
 "R_Women's Featherweight_rank",
 'B_Pound-for-Pound_rank',
 "B_Women's Flyweight_rank",
 "R_Women's Flyweight_rank",
 "B_Women's Strawweight_rank",
 "B_Women's Bantamweight_rank",
 'B_Bantamweight_rank',
 'B_Lightweight_rank',
 'B_Welterweight_rank',
 'B_Featherweight_rank',
 'B_Light Heavyweight_rank',
 'B_Flyweight_rank',
 'B_Middleweight_rank',
 "R_Women's Strawweight_rank",
 'B_Heavyweight_rank',
 "R_Women's Bantamweight_rank",
 'R_Featherweight_rank',
 'R_Middleweight_rank',
 'R_Bantamweight_rank',
 'R_Lightweight_rank',
 'R_Welterweight_rank',
 'R_Light Heavyweight_rank',
 'R_Heavyweight_rank',
 'R_Flyweight_rank',
 'R_Pound-for-Pound_rank',
 'B_match_weightclass_rank',
 'R_match_weightclass_rank',
 'finish_details']

In [19]:
# Dropping the selected columns
df.drop(columns_to_drop, axis=1, inplace=True)

In [20]:
# Looking of list of columns with the most null values
nulls2 = df.isna().sum()
nulls2.sort_values(ascending=False)

b_ko_odds                      1062
b_sub_odds                     1061
r_ko_odds                      1049
r_sub_odds                     1049
B_avg_SIG_STR_landed            930
                               ... 
R_win_by_Decision_Split           0
R_win_by_Decision_Unanimous       0
R_win_by_KO/TKO                   0
R_win_by_Submission               0
R_fighter                         0
Length: 90, dtype: int64

I was comfortable dropping the rest of the nulls in the dataset because I still had a lot of data to work with and I thought these columns with the most nulls would be vital features for modeling.

In [21]:
# Dropping the rest of the null values
clean_df = df.dropna()
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2615 entries, 0 to 4286
Data columns (total 90 columns):
 #   Column                        Non-Null Count  Dtype         
---  ------                        --------------  -----         
 0   R_fighter                     2615 non-null   object        
 1   B_fighter                     2615 non-null   object        
 2   R_odds                        2615 non-null   float64       
 3   B_odds                        2615 non-null   int64         
 4   R_ev                          2615 non-null   float64       
 5   B_ev                          2615 non-null   float64       
 6   date                          2615 non-null   datetime64[ns]
 7   location                      2615 non-null   object        
 8   country                       2615 non-null   object        
 9   Winner                        2615 non-null   object        
 10  title_bout                    2615 non-null   bool          
 11  weight_class                  

I needed to know if the winner of the fight is the underdog or the favorite. The data had this column listed as the 'Red' or 'Blue' fighter. I used the `B_odds` and `R_odds` to discover which fighter the underdog was. Then created two new columns `underdog` and `favored`. These contained the names of the fighters. Then was able to replace 'Red' or 'Blue' in the `Winner` column with either 'underdog' or 'favored'.

In [22]:
clean_df['Winner'].value_counts()

Red     1513
Blue    1102
Name: Winner, dtype: int64

In [23]:
# Creating a column that has the name of the underdog by looking at the odds columns
clean_df['underdog'] = clean_df.apply(
    lambda row: row['R_fighter'] if row['R_odds'] > row['B_odds'] else row['B_fighter'], axis=1)
# Creating a column that has the name of the favored fighter by looking at the odds columns
clean_df['favored'] = clean_df.apply(
    lambda row: row['R_fighter'] if row['R_odds'] < row['B_odds'] else row['B_fighter'], axis=1)

<ipython-input-23-f1a5e82c79dc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_df['underdog'] = clean_df.apply(
<ipython-input-23-f1a5e82c79dc>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_df['favored'] = clean_df.apply(


In [24]:
# Changing Red winners to either underdog or favored
clean_df.loc[(clean_df['Winner'] == 'Red') & (clean_df['R_fighter'] == clean_df['underdog']), 'Winner'] = 'underdog'
clean_df.loc[(clean_df['Winner'] == 'Red') & (clean_df['R_fighter'] != clean_df['underdog']), 'Winner'] = 'favored'
# Changing Blue winners to either underdog or favored
clean_df.loc[(clean_df['Winner'] == 'Blue') & (clean_df['B_fighter'] == clean_df['underdog']), 'Winner'] = 'underdog'
clean_df.loc[(clean_df['Winner'] == 'Blue') & (clean_df['B_fighter'] != clean_df['underdog']), 'Winner'] = 'favored'

C:\Users\42ben\anaconda3\envs\learn-env\lib\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [25]:
clean_df['Winner'].value_counts(normalize=True)

favored     0.640918
underdog    0.359082
Name: Winner, dtype: float64

Because I got rid of a lot of rows due to them containing null values I wanted to see if I created a larger class imbalance than what already existed within the whole dataset. After looking at the full dataset it contains almost the exact same percentage of underdog and favored winners.

In [26]:
# Dropping any nulls that would affect making the changes made below
df['R_odds'].dropna(inplace=True)
df['B_odds'].dropna(inplace=True)
df['R_fighter'].dropna(inplace=True)
df['B_fighter'].dropna(inplace=True)

# Creating a column that has the name of the underdog by looking at the odds columns
df['underdog'] = df.apply(
    lambda row: row['R_fighter'] if row['R_odds'] > row['B_odds'] else row['B_fighter'], axis=1)
# Creating a column that has the name of the favored fighter by looking at the odds columns
df['favored'] = df.apply(
    lambda row: row['R_fighter'] if row['R_odds'] < row['B_odds'] else row['B_fighter'], axis=1)

# Changing Red winners to either underdog or favored
df.loc[(df['Winner'] == 'Red') & (df['R_fighter'] == df['underdog']), 'Winner'] = 'underdog'
df.loc[(df['Winner'] == 'Red') & (df['R_fighter'] != df['underdog']), 'Winner'] = 'favored'
# Changing Blue winners to either underdog or favored
df.loc[(df['Winner'] == 'Blue') & (df['B_fighter'] == df['underdog']), 'Winner'] = 'underdog'
df.loc[(df['Winner'] == 'Blue') & (df['B_fighter'] != df['underdog']), 'Winner'] = 'favored'

df['Winner'].value_counts(normalize=True)

favored     0.652982
underdog    0.347018
Name: Winner, dtype: float64

Running a .corr() in order to see which features are the most correlated with the target column. After inspecting the table these are the features I decided to use for my first models:
- `B_odds`
- `R_odds`
- `B_win_by_Decision_Split`
- `B_win_by_Decision_Unanimous`
- `win_streak_dif`
- `loss_dif`
- `reach_dif`
- `age_dif`
- `r_dec_odds`
- `b_dec_odds`
- `b_sub_odds`
- `r_ko_odds`
- `b_ko_odds`

In [27]:
# Creating a copy of clean_df
correlation = clean_df.copy()
# Changing 'Winner' to numeric data
correlation['Winner'] = correlation['Winner'].map({'favored': 0, 'underdog': 1})
# Changing Stance columns for each fighter to numeric data
correlation['R_Stance'] = correlation['R_Stance'].map({'Orthodox': 0, 'Southpaw': 1, 'Switch': 2})
correlation['B_Stance'] = correlation['B_Stance'].map({'Orthodox': 0, 'Southpaw': 1, 'Switch': 2})
# Running .corr on table
correlation.corr()

,R_odds,B_odds,R_ev,B_ev,Winner,title_bout,no_of_rounds,B_current_lose_streak,B_current_win_streak,B_draw,...,empty_arena,constant_1,finish_round,total_fight_time_secs,r_dec_odds,b_dec_odds,r_sub_odds,b_sub_odds,r_ko_odds,b_ko_odds
R_odds,1.000000,-0.969696,0.820485,-0.936470,0.137393,-0.193818,-0.157417,-0.027434,0.024014,0.010812,...,-0.000842,NaN,-0.036908,-0.019921,0.356514,-0.675676,0.324984,-0.392236,0.508780,-0.601083
B_odds,-0.969696,1.000000,-0.878323,0.892632,-0.120096,0.174785,0.152741,0.036489,-0.035938,-0.006048,...,-0.009896,NaN,0.035872,0.020138,-0.395906,0.640760,-0.338724,0.382485,-0.531917,0.586146
R_ev,0.820485,-0.878323,1.000000,-0.639952,0.029789,-0.116346,-0.117465,-0.043147,0.046130,-0.008782,...,-0.028255,NaN,-0.053456,-0.048283,0.463268,-0.466056,0.345334,-0.303920,0.532878,-0.476202
B_ev,-0.936470,0.892632,-0.639952,1.000000,-0.177425,0.214087,0.171286,0.010087,-0.008161,-0.024703,...,-0.039891,NaN,0.019696,-0.002098,-0.267131,0.715850,-0.281088,0.397651,-0.445476,0.596425
Winner,0.137393,-0.120096,0.029789,-0.177425,1.000000,-0.033924,0.004655,0.003142,0.008900,0.030676,...,-0.022019,NaN,0.001515,0.000013,0.034843,-0.098497,-0.004560,-0.057562,0.058426,-0.118841
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
b_dec_odds,-0.675676,0.640760,-0.466056,0.715850,-0.098497,0.264454,0.307118,-0.013570,0.020783,-0.044774,...,-0.034307,NaN,-0.016122,-0.055197,0.074246,1.000000,-0.194874,0.300780,-0.483733,0.190382
r_sub_odds,0.324984,-0.338724,0.345334,-0.281088,-0.004560,-0.028571,0.023407,-0.006109,0.055771,0.022911,...,0.017766,NaN,0.020821,0.021710,0.141541,-0.194874,1.000000,-0.315952,-0.074263,-0.132230
b_sub_odds,-0.392236,0.382485,-0.303920,0.397651,-0.057562,0.143187,0.154065,-0.002178,0.048102,-0.003676,...,0.022419,NaN,0.034464,0.017958,-0.070957,0.300780,-0.315952,1.000000,-0.117192,0.024339
r_ko_odds,0.508780,-0.531917,0.532878,-0.445476,0.058426,-0.149433,-0.227844,-0.032786,-0.004365,0.035661,...,-0.030776,NaN,0.018983,0.046386,-0.059935,-0.483733,-0.074263,-0.117192,1.000000,-0.211696


I ran .value_counts() on all columns to make sure all data looked valid. 

In [28]:
clean_df['no_of_rounds'].value_counts()

3    2294
5     320
4       1
Name: no_of_rounds, dtype: int64

Dropped this row because there is no such thing as a 4 round fight. They are either 3 or 5 rounds long.

In [29]:
# Dropping the row
clean_df.drop(clean_df.loc[clean_df['no_of_rounds'] == 4].index, inplace=True)

C:\Users\42ben\anaconda3\envs\learn-env\lib\site-packages\pandas\core\frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


Completed data cleaning and performed a train/test split. All features were numeric.

In [30]:
# Setting all wanted features equal to X
X = clean_df[['B_odds', 'R_odds', 'B_win_by_Decision_Split', 'B_win_by_Decision_Unanimous',
                    'win_streak_dif', 'loss_dif', 'reach_dif', 'age_dif', 'r_dec_odds', 'b_dec_odds',
                  'b_sub_odds', 'r_ko_odds', 'b_ko_odds']]
# Setting target equal to y
y = clean_df['Winner']
# Performing the train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=333)

## Modeling

## Evaluation

A dummy model would pick the underdog every time and be correct about 36% of the time. This also displays an expected yet slight class imbalance, nothing dramatic.

In [31]:
y_train.value_counts(normalize=True)

favored     0.637245
underdog    0.362755
Name: Winner, dtype: float64

### First Model (Random Forest)
For my first model created a random forest with all default hyperparameters to see if it would over fit to the training data and score a perfect 1.0 

In [41]:
# Creating steps for a Pipeline
forest_steps = [('scaler', StandardScaler()),
             ('forest', RandomForestClassifier(random_state=333))]
# Creating Pipeline with steps
forest_pipe = Pipeline(forest_steps)
# Fitting the training data to the Pipeline
forest_pipe.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('forest', RandomForestClassifier(random_state=333))])

It did get a perfect score on the training data which gives me confidence I'm working with data that is capable of creating a sufficient model.

In [42]:
# Defining the predictions from the Pipeline using the training data
y_pred = forest_pipe.predict(X_train)
# Evaluating the accuracy score on the training data
accuracy_score(y_train, y_pred), precision_score(y_train, y_pred, pos_label='underdog')

(1.0, 1.0)

The cross validation accuracy score is pretty good for a first model with all default hyperparameters. It is about 20% higher than the dummy model.

In [43]:
# Calculating mean of the 5-fold cross_val_score on accuracy
cross_val_score(tree_pipe, X_train, y_train, cv=5, scoring='accuracy').mean()

0.5688775510204082

I made a custom scorer for precision to use when cross validating because this is the metric I'm mostly concerned with.

In [44]:
# Making a custom scorer for precision
precision_scorer = make_scorer(precision_score, pos_label='underdog')

This precision score is also not too bad for the first model at 41%

In [45]:
# Calculating mean of the 5-fold cross_val_score on precision
cross_val_score(tree_pipe, X_train, y_train, cv=5, scoring=precision_scorer).mean()

0.4092745140796266

### Random Forest 2nd iteration (Using GridSearch)

Created a GridSearch to try and improve precision score by tweaking the hyperparameters of the random forest. It is overfit so limiting the max_depth was a priority. There is a slight class imbalance so I tried changing the weights. Criterion and n_estimators were also of interest.

In [46]:
# Creating parameters for GridSearch
forest_params = {'forest__n_estimators': [50, 100, 150],
                'forest__max_depth': [5, 50, 100],
                'forest__class_weight': ['balanced', None],
                'forest__criterion': ['gini', 'entropy', 'log_loss']}
# GridSearch with the random forest pipeline, parameters above, 5 fold cross validation, and precision score
forest_grid = GridSearchCV(estimator=forest_pipe, param_grid=forest_params, cv=5, scoring=precision_scorer, n_jobs=3)
# Fitting the GridSearch
forest_grid.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('forest',
                                        RandomForestClassifier(random_state=333))]),
             n_jobs=3,
             param_grid={'forest__class_weight': ['balanced', None],
                         'forest__criterion': ['gini', 'entropy', 'log_loss'],
                         'forest__max_depth': [5, 50, 100],
                         'forest__n_estimators': [50, 100, 150]},
             return_train_score=True,
             scoring=make_scorer(precision_score, pos_label=underdog))

The GridSearch found n_estimators=150, max_depth=50, default class weights, and criterion='entropy' returned the best precision score. It improved quite dramtically for the first GridSearch. Precision went up by approximately 8%. 

In [47]:
# Displaying the best estimator from the GridSearch and the precision score
print(forest_grid.best_estimator_)
print(forest_grid.best_score_)

Pipeline(steps=[('scaler', StandardScaler()),
                ('forest',
                 RandomForestClassifier(criterion='entropy', max_depth=50,
                                        n_estimators=150, random_state=333))])
0.49200305906061487


This model is still very overfit to the training data.

In [48]:
# Displaying precision score on the training data
forest_grid.best_estimator_.score(X_train, y_train)

1.0

### Random Forest 3rd iteration (Using GridSearch)

Tried to improve results with a new GridSearch based on the results from the previous GridSearch.

In [50]:
# Creating parameters for GridSearch
forest_params2 = {'forest__n_estimators': [140, 150, 160],
                 'forest__max_depth': [45, 50, 75],
                 'forest__class_weight': [{'favored': 1, 'underdog': 2}, None],
                 'forest__criterion': ['gini', 'entropy', 'log_loss']}
# GridSearch with the random forest pipeline, parameters above, 5 fold cross validation, and precision score
forest_grid2 = GridSearchCV(estimator=forest_pipe, param_grid=forest_params2, cv=5, scoring=precision_scorer, n_jobs=3)
# Fitting the GridSearch
forest_grid2.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('forest',
                                        RandomForestClassifier(random_state=333))]),
             n_jobs=3,
             param_grid={'forest__class_weight': [{'favored': 1, 'underdog': 2},
                                                  None],
                         'forest__criterion': ['gini', 'entropy', 'log_loss'],
                         'forest__max_depth': [45, 50, 75],
                         'forest__n_estimators': [140, 150, 160]},
             scoring=make_scorer(precision_score, pos_label=underdog))

The only difference from this GridSearch was it picked a max_depth of 45 instead of 50 but the results were virtually the same.

In [51]:
# Displaying the best estimator from the GridSearch and the precision score
print(forest_grid2.best_estimator_)
print(forest_grid2.best_score_)

Pipeline(steps=[('scaler', StandardScaler()),
                ('forest',
                 RandomForestClassifier(criterion='entropy', max_depth=45,
                                        n_estimators=150, random_state=333))])
0.49200305906061487


It is still very overfit and scoring perfectly on the training data.

In [53]:
# Displaying precision score on the training data
forest_grid2.best_estimator_.score(X_train, y_train)

1.0

### Random Forest with PCA
One way to help with an overfit model is to reduce dimensionality. I decided to do this by adding PCA into a new pipeline for a random forest model.

In [56]:
# Creating steps for a Pipeline w/ PCA
pca_forest_steps = [('scaler', StandardScaler()),
                    ('pca', PCA()),
                    ('forest', RandomForestClassifier(random_state=333))]
# Creating Pipeline with steps
pca_forest_pipe = Pipeline(pca_forest_steps)
# Fitting the training data to the Pipeline
pca_forest_pipe.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()), ('pca', PCA()),
                ('forest', RandomForestClassifier(random_state=333))])

I set the GridSearch to search through a wide variety of hyperparameters. This included defualt settings and what worked well for the random forest without PCA plus some other values to see what works best now that PCA is being used. For n_components in PCA I used a very wide range because I didn't know how much I should reduce the complexity.

In [57]:
# Creating parameters for GridSearch w/ PCA
forest_params3 = {'pca__n_components': [2, 0.8, 0.9],
                  'forest__n_estimators': [50, 100, 150],
                  'forest__max_depth': [5, 50, 100],
                  'forest__class_weight': ['balanced', None],
                  'forest__criterion': ['gini', 'entropy', 'log_loss']}
# GridSearch with the random forest pipeline, parameters above, 5 fold cross validation, and precision score
forest_grid3 = GridSearchCV(estimator=pca_forest_pipe, param_grid=forest_params3, cv=5, scoring=precision_scorer, n_jobs=3)
# Fitting the GridSearch
forest_grid3.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('pca', PCA()),
                                       ('forest',
                                        RandomForestClassifier(random_state=333))]),
             n_jobs=3,
             param_grid={'forest__class_weight': ['balanced', None],
                         'forest__criterion': ['gini', 'entropy', 'log_loss'],
                         'forest__max_depth': [5, 50, 100],
                         'forest__n_estimators': [50, 100, 150],
                         'pca__n_components': [2, 0.8, 0.9]},
             scoring=make_scorer(precision_score, pos_label=underdog))

This increased the precision score by about 11%! It now earned a precision score of 60%

In [58]:
# Displaying the best estimator from the GridSearch and the precision score
print(forest_grid3.best_estimator_)
print(forest_grid3.best_score_)

Pipeline(steps=[('scaler', StandardScaler()), ('pca', PCA(n_components=2)),
                ('forest',
                 RandomForestClassifier(criterion='entropy', max_depth=5,
                                        random_state=333))])
0.6


This model is much less overfit than the random forest models without PCA. It is still slightly overfit.

In [59]:
# Displaying precision score on the training data
forest_grid3.best_estimator_.score(X_train, y_train)

0.6454081632653061

### Random Forest with PCA 2nd Iteration
Updated pipeline with hyperparamers that have consistently been the best. Criterion='entropy' and default class weight.

In [60]:
# Creating steps for a Pipeline w/ PCA
pca_forest_steps = [('scaler', StandardScaler()),
                    ('pca', PCA()),
                    ('forest', RandomForestClassifier(random_state=333, criterion='entropy'))]
# Creating Pipeline with steps
pca_forest_pipe = Pipeline(pca_forest_steps)
# Fitting the training data to the Pipeline
pca_forest_pipe.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()), ('pca', PCA()),
                ('forest',
                 RandomForestClassifier(criterion='entropy',
                                        random_state=333))])

Tried to increase the results with new GridSearch. No longer using GridSearch for criterion or class weight.

In [61]:
# Creating parameters for GridSearch w/ PCA
forest_params4 = {'pca__n_components': [2, 3],
                  'forest__n_estimators': [99, 100, 101],
                  'forest__max_depth': [4, 5, 6]}
# GridSearch with the random forest pipeline, parameters above, 5 fold cross validation, and precision score
forest_grid4 = GridSearchCV(estimator=pca_forest_pipe, param_grid=forest_params4, cv=5, scoring=precision_scorer, n_jobs=3)
# Fitting the GridSearch
forest_grid4.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('pca', PCA()),
                                       ('forest',
                                        RandomForestClassifier(criterion='entropy',
                                                               random_state=333))]),
             n_jobs=3,
             param_grid={'forest__max_depth': [4, 5, 6],
                         'forest__n_estimators': [99, 100, 101],
                         'pca__n_components': [2, 3]},
             scoring=make_scorer(precision_score, pos_label=underdog))

This GridSearch returned slightly different hyperparameters but virtually the same scores. 

In [62]:
# Displaying the best estimator from the GridSearch and the precision score
print(forest_grid4.best_estimator_)
print(forest_grid4.best_score_)

Pipeline(steps=[('scaler', StandardScaler()), ('pca', PCA(n_components=3)),
                ('forest',
                 RandomForestClassifier(criterion='entropy', max_depth=4,
                                        n_estimators=99, random_state=333))])
0.6


This was marginally more overfit than the previous model.

In [63]:
forest_grid4.best_estimator_.score(X_train, y_train)

0.6464285714285715